In [17]:
#Import different libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
import pickle


In [18]:
#Load the model
model = load_model('models/cnn_model.h5')

In [19]:
#Import paths
BASE_PATH = "../"
NEW_DATA_TRAIN = os.path.join(BASE_PATH, "new_data/train")

In [20]:
#Define parameters
img_height, img_width = 150, 150
batch_size = 32
epochs = 30

In [21]:
#Prepare the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    NEW_DATA_TRAIN,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    classes=['covid', 'lung_opacity', 'normal', 'pneumonia']
)


Found 16 images belonging to 4 classes.


In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
#Set up callbacks
callbacks = [
    EarlyStopping(monitor='loss', patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath='models/cnn_model.keras', save_best_only=True)
]


In [30]:
#Retrain the model with new data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=train_generator,
    callbacks=callbacks
)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 1.2852e-04 - val_accuracy: 1.0000 - val_loss: 1.1307e-04
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 8.3765e-05
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 1.1535e-04 - val_accuracy: 1.0000 - val_loss: 6.4010e-05
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 1.0000 - val_loss: 5.1286e-05
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 4.3121e-05
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 1.5178e-04 - val_accuracy: 1.0000 - val_loss: 3.6692e-05


In [31]:
#Evaluate the model
loss, accuracy = model.evaluate(train_generator, steps=train_generator.samples // train_generator.batch_size)
print(f"Training accuracy: {accuracy * 100:.2f}%")

#Save the updated model
model.save('models/cnn_model_incremental.h5')

#Convert model to pickle
with open('models/cnn_model_incremental.pkl', 'wb') as file:
    pickle.dump(model, file)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 6.4010e-05


Training accuracy: 100.00%
